<a href="https://colab.research.google.com/github/tom5610/my-fsdl-text-recognizer-2022/blob/main/lab01/notebooks/c_lab01_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lab 01: Deep Neural Networks in PyTorch

Objective:
* how to write a basic neural network from scatch in PyTorch
* How the submodules of torch, like `torch.nn` and `torch.utils.data`, make writing performant neural network training and inference code easier

## Setup
Below cell will run full environment setup in colab.

In [23]:
import os
from pathlib import Path
cwd = Path.cwd()
print(cwd)

lab_idx = 1
lab_name = f"lab{str(lab_idx).zfill(2)}"
my_fsdl = "my-fsdl-text-recognizer-2022"
os.chdir(f"{cwd}/{my_fsdl}/{lab_name}")

Path.cwd()

/Users/tomlu/Workspace


PosixPath('/Users/tomlu/Workspace/my-fsdl-text-recognizer-2022/lab01')

In [1]:
lab_idx = 1

if 'bootstrap' not in locals() or bootstrap.run:
    pythonpath, = !echo $PYTHONPATH
    if '.' not in pythonpath.split(':'):
        pythonpath = ".:" + pythonpath
        %env PYTHONPATH={pythonpath}
        !echo $PYTHONPATH

    # get both Colab and local notebooks into the same state
    !wget --quiet https://fsdl.me/gist-bootstrap -O bootstrap.py
    import bootstrap

    bootstrap.change_to_lab_dir(lab_idx=lab_idx)

    # allow 'hot-reloading' of modules
    %load_ext autoreload
    %autoreload 2
    # needed for inline plots in some contexts
    %matplotlib inline

    bootstrap.run = False

!pwd
%ls

env: PYTHONPATH=.:/env/python
.:/env/python
/content/fsdl-text-recognizer-2022-labs/lab01
notebooks/  text_recognizer/


## Getting data and making Tensors

before we can build a model, we need data. 

Use MNIST dataset of handwritten digits.

The data used to train state-of-the-art models these days is generally too large to be stored on the disk of any single machine (to say nothing of the RAM!), so fetching data over a network is a common first step in model training.

In [2]:
 # allow 'hot-reloading' of modules
%load_ext autoreload
%autoreload 2
# needed for inline plots in some contexts
%matplotlib inline


In [3]:
from pathlib import Path 
import requests

def download_mnist(path):
    url = "https://github.com/pytorch/tutorials/raw/master/_static/"
    filename = "mnist.pkl.gz"

    if not (path / filename).exists():
        content = requests.get(url + filename).content
        (path / filename).open("wb").write(content)

    return path / filename

data_path = Path("data") if Path("data").exists() else Path("../data")
path = data_path / "downloaded" / "vector-mnist"
path.mkdir(parents=True, exist_ok=True)

datafile = download_mnist(path)

In [4]:
import gzip
import pickle 

def read_mnist(path):
    with gzip.open(path, "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")
    return x_train, y_train, x_valid, y_valid

x_train, y_train, x_valid, y_valid = read_mnist(datafile)
    

In [5]:
import torch 

x_train, y_train, x_valid, y_valid = map(
    torch.tensor, (x_train, y_train, x_valid, y_valid)
)

In [6]:
print(x_train, y_train, sep="\n")

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([5, 0, 4,  ..., 8, 4, 8])


In [8]:
y_train[0], x_train[0, ::2]

(tensor(5),
 tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0117, 0.0703, 0.4922, 0.6836, 0.6484,
         0.9648, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1172, 0.3672,
         0.6641, 0.9883, 0.9883, 0.8789, 0.9883, 0.7617, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.9297, 0.9883, 0.9883, 0.9883, 0.9883, 0.3633,
         0.3203,

In [7]:
x_train.ndim, y_train.ndim

(2, 1)

In [8]:
n, c = x_train.shape
print(x_train.shape)
print(y_train.shape)

torch.Size([50000, 784])
torch.Size([50000])


In [16]:
import random 

import wandb 

import text_recognizer.metadata.mnist as metadata

idx = random.randint(0, len(x_train))
example = x_train[idx]

print(y_train[idx])
wandb.Image(example.reshape(*metadata.DIMS)).image

ModuleNotFoundError: No module named 'text_recognizer'

## Build a DNN using only `torch.Tensor` methods and Python

### Defining the model

In [10]:
import math

import torch
weights = torch.randn(784, 10) / math.sqrt(784)
weights.requires_grad_()
bias = torch.zeros(10, requires_grad=True)

In [11]:
weights.ndim, weights.shape

(2, torch.Size([784, 10]))

In [12]:
def linear(x: torch.Tensor) -> torch.Tensor:
    return x @ weights + bias

In [13]:
n = 3
a = torch.rand(n)
b = torch.rand(n)
print(torch.matmul(a, b))
print(a @ b)
a, b

tensor(0.5742)
tensor(0.5742)


(tensor([0.6208, 0.2513, 0.5182]), tensor([0.0318, 0.3594, 0.8958]))

In [14]:
0.5367 * 0.0799 + 0.361 * 0.3948 + 0.3556 * 0.6106

0.40253448999999997

In [15]:
a = torch.rand((3, 2))
b = torch.rand((2, 2))
result = a.mm(b)

In [16]:
a, b, result

(tensor([[0.2096, 0.0723],
         [0.4111, 0.4794],
         [0.4698, 0.5032]]), tensor([[0.3150, 0.3239],
         [0.4464, 0.0986]]), tensor([[0.0983, 0.0750],
         [0.3435, 0.1804],
         [0.3726, 0.2018]]))

In [17]:
def log_softmax(x: torch.Tensor) -> torch.Tensor:
    return x - torch.log(torch.sum(torch.exp(x), axis = 1))[:, None]

def model(xb: torch.Tensor) -> torch.Tensor:
    return log_softmax(linear(xb))

In [18]:
bs = 64 # batch size

xb = x_train[0: bs] # a batch of inputs
outs = model(xb)

print(outs[0], outs.shape)


tensor([-2.9019, -2.7442, -2.6731, -2.3839, -2.5940, -1.9546, -1.7174, -2.4605,
        -2.4805, -1.8670], grad_fn=<SelectBackward0>) torch.Size([64, 10])


## Defining the loss and metrics

In [107]:
def accuracy(out: torch.Tensor, yb: torch.Tensor) -> torch.Tensor:
    preds = torch.argmax(out, dim=1)
    #print('accuracy comparison - ', (preds == yb).float())
    return (preds == yb).float().mean()

In [20]:
yb = y_train[0: bs]

acc = accuracy(outs, yb)

print(acc)

accuracy comparison -  tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 1.])
tensor(0.1094)


In [21]:
def cross_entropy(output: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    return -output[range(target.shape[0]), target].mean()

loss_func = cross_entropy

In [22]:
print(loss_func(outs, yb), -torch.log(torch.tensor(1 / 10)))

tensor(2.2519, grad_fn=<NegBackward0>) tensor(2.3026)


In [23]:
torch.log(torch.tensor(1 / 10))

tensor(-2.3026)

In [24]:
loss = loss_func(outs, yb)

loss.backward()

In [25]:
weights.grad, bias.grad

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([-0.0011, -0.0529,  0.0256, -0.0021, -0.0255,  0.0595,  0.0155, -0.0051,
          0.0027, -0.0166]))

## Defining and running the fitting loop

We now have all the ingredients we need to fit a neural network to data:

* data (x_train, y_train)
* a network architecture with parameters (model, weights, and bias)
* a loss_func to optimize (cross_entropy) that supports .backward computation of gradients

In [26]:
lr = 0.5
epochs = 2

for epoch in range(epochs): # loop over the data repeatedly
    for ii in range((n - 1) // bs + 1):
        start_idx = ii * bs
        end_idx = start_idx + bs

        # pull batches from x and from y
        xb = x_train[start_idx:end_idx]
        yb = y_train[start_idx:end_idx]

        # run model
        pred = model(xb)

        # get loss
        loss = loss_func(pred, yb)

        # calculate the gradients with a backwards pass
        loss.backward()

        # update the parameters
        with torch.no_grad(): # we don't want to track gradients through this part!
            # SGD learning rule: update with negative gradient scaled by lr
            weights -= weights.grad * lr
            bias -= bias.grad * lr

            # achtung: PyTorch doesn't assume you're done with gradients
            #.         until you say so -- by explicitly "deleting" them. i.e. setting the gradients to 0.
            weights.grad.zero_()
            bias.grad.zero_()


In [27]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))

accuracy comparison -  tensor([0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
        1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
tensor(0.9472, grad_fn=<NegBackward0>) tensor(0.8281)


In [28]:
idx = random.randint(0, len(x_train))
example = x_train[idx: idx+1]

out = model(example)

print(out.argmax())
wandb.Image(example.reshape(28, 28)).image

tensor(4)


## Refactoring with core `torch.nn` components


### Using `torch.nn.functional` for stateless computation

In [29]:
import torch.nn.functional as F

loss_func = F.cross_entropy

def model(xb):
    return xb @ weights + bias

In [30]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))

accuracy comparison -  tensor([0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
        1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
tensor(0.9472, grad_fn=<NllLossBackward0>) tensor(0.8281)


### Using torch.nn.Module to define functions whose state is given by `torch.nn

In [31]:
from torch import nn

class MNISTLogistic(nn.Module):
    def __init__(self):
        super().__init__() # this is mandatory as nn.Module does import setup
        self.weights = nn.Parameter(torch.randn(784, 10) / math.sqrt(784))
        self.bias = nn.Parameter(torch.zeros(10))

In [32]:
def forward(self, xb:torch.Tensor) -> torch.Tensor:
    return xb @ self.weights + self.bias

MNISTLogistic.forward = forward

model = MNISTLogistic()
print(model(xb)[:4])
loss = loss_func(model(xb), yb)
loss.backward()
print(model.weights.grad[::17, ::2])

tensor([[-0.0634,  0.6026,  0.0680, -0.0123, -0.1119, -0.5882, -0.2499, -0.0625,
         -0.1788, -0.3474],
        [ 0.4817,  0.4657,  0.4165,  0.3899, -0.2445, -0.2347,  0.2087, -0.5309,
         -0.8083, -0.4876],
        [-0.0350,  0.2182,  0.2181,  0.1475, -0.1530,  0.2505, -0.5652, -0.1470,
          0.0150, -0.0387],
        [ 0.3391,  0.2915, -0.2129,  0.1553,  0.1105,  0.1131,  0.1531,  0.2332,
         -0.3465,  0.3848]], grad_fn=<SliceBackward0>)
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0033,  0.0038,  0.0024, -0.0262,  0.0029],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0044, -0.0054,  0.0255, -0.022

In [33]:
print(*list(model.parameters()), sep="\n")

Parameter containing:
tensor([[-0.0131, -0.0375,  0.0346,  ..., -0.0007, -0.0096, -0.0604],
        [-0.0717, -0.0018,  0.0351,  ...,  0.0579,  0.0178,  0.0087],
        [-0.0325, -0.0509, -0.0461,  ..., -0.0065,  0.0325,  0.0158],
        ...,
        [ 0.0365,  0.0202,  0.0309,  ..., -0.0218,  0.0219, -0.0927],
        [-0.0489,  0.0212,  0.0028,  ..., -0.0283, -0.0154, -0.0650],
        [-0.0240, -0.0796, -0.0292,  ...,  0.0144,  0.0453,  0.0060]],
       requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)


In [34]:
def fit():
    for epoch in range(epochs):
        for ii in range((n - 1) // bs + 1):
            start_idx = ii * bs 
            end_idx = start_idx + bs
            yb = x_train[start_idx: end_idx]
            yb = y_train[start_idx: end_idx]
            pred = model(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            with torch.no_grad():
                for p in model.parameters():
                    p -= p.grad * lr
                model.zero_grad()

fit()

In [35]:
print(accuracy(model(xb), yb))

accuracy comparison -  tensor([0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1.,
        1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
        0., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
tensor(0.8125)


In [36]:
import textwrap 

print("torch.nn.Modules:", *textwrap.wrap(", ".join(torch.nn.modules.__all__)), sep="\n\t")


torch.nn.Modules:
	Module, Identity, Linear, Conv1d, Conv2d, Conv3d, ConvTranspose1d,
	ConvTranspose2d, ConvTranspose3d, Threshold, ReLU, Hardtanh, ReLU6,
	Sigmoid, Tanh, Softmax, Softmax2d, LogSoftmax, ELU, SELU, CELU, GLU,
	GELU, Hardshrink, LeakyReLU, LogSigmoid, Softplus, Softshrink,
	MultiheadAttention, PReLU, Softsign, Softmin, Tanhshrink, RReLU,
	L1Loss, NLLLoss, KLDivLoss, MSELoss, BCELoss, BCEWithLogitsLoss,
	NLLLoss2d, PoissonNLLLoss, CosineEmbeddingLoss, CTCLoss,
	HingeEmbeddingLoss, MarginRankingLoss, MultiLabelMarginLoss,
	MultiLabelSoftMarginLoss, MultiMarginLoss, SmoothL1Loss,
	GaussianNLLLoss, HuberLoss, SoftMarginLoss, CrossEntropyLoss,
	Container, Sequential, ModuleList, ModuleDict, ParameterList,
	ParameterDict, AvgPool1d, AvgPool2d, AvgPool3d, MaxPool1d, MaxPool2d,
	MaxPool3d, MaxUnpool1d, MaxUnpool2d, MaxUnpool3d, FractionalMaxPool2d,
	FractionalMaxPool3d, LPPool1d, LPPool2d, LocalResponseNorm,
	BatchNorm1d, BatchNorm2d, BatchNorm3d, InstanceNorm1d, InstanceNorm2d,

In [37]:
class MNISTLogistic(nn.Module):
    def __init__(self):
        super().__init__() 
        self.lin = nn.Linear(784, 10)

    def forward(self, xb):
        return self.lin(xb)

In [38]:
model = MNISTLogistic()
print(loss_func(model(xb), yb))

tensor(2.3073, grad_fn=<NllLossBackward0>)


In [39]:
print(*list(model.children()))

Linear(in_features=784, out_features=10, bias=True)


In [40]:
print(*list(model.parameters()), sep="\n")

Parameter containing:
tensor([[-0.0244, -0.0078,  0.0233,  ...,  0.0039,  0.0260, -0.0055],
        [-0.0046, -0.0072,  0.0091,  ..., -0.0174,  0.0290, -0.0111],
        [-0.0271, -0.0218, -0.0245,  ...,  0.0295, -0.0101,  0.0011],
        ...,
        [ 0.0275,  0.0072, -0.0120,  ...,  0.0153, -0.0182,  0.0048],
        [ 0.0167,  0.0322,  0.0059,  ..., -0.0221, -0.0267, -0.0319],
        [-0.0107, -0.0088,  0.0277,  ...,  0.0231, -0.0083, -0.0141]],
       requires_grad=True)
Parameter containing:
tensor([ 0.0131, -0.0102, -0.0111,  0.0134,  0.0346,  0.0006,  0.0306, -0.0313,
         0.0172,  0.0356], requires_grad=True)


### Applying gradients with `torch.optim.Optimizer`

In [41]:
from torch import optim 

def configure_optimizer(model: nn.Module) -> optim.Optimizer:
    return optim.Adam(model.parameters(), lr=3e-4)

In [42]:
model = MNISTLogistic()
opt = configure_optimizer(model)

for epoch in range(epochs):
    for ii in range((n - 1) // bs + 1):
        start_idx = ii * bs 
        end_idx = start_idx + bs 
        xb = x_train[start_idx: end_idx]
        yb = y_train[start_idx: end_idx]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print("after training: ", loss_func(model(xb), yb), sep="\n\t")

after training: 
	tensor(2.3011, grad_fn=<NllLossBackward0>)


In [43]:
from text_recognizer.data.util import BaseDataset 

train_ds = BaseDataset(x_train, y_train)

In [44]:
BaseDataset??

In [45]:
model = MNISTLogistic()
opt = configure_optimizer(model)

for epoch in range(epochs):
    for ii in range((n - 1) // bs + 1):
        xb, yb = train_ds[ii * bs: ii * bs + bs]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print(loss_func(model(xb), yb))

tensor(2.2878, grad_fn=<NllLossBackward0>)


### Batching up data with `torch.utils.data.DataLoader`

In [46]:
from torch.utils.data import DataLoader

train_ds = BaseDataset(x_train, y_train)
train_dataloader = DataLoader(train_ds, batch_size = bs)

In [47]:
def fit(self: nn.Module, train_dataloader: DataLoader):
    opt = configure_optimizer(self)

    for epoch in range(epochs):
        for xb, yb in train_dataloader:
            pred = self(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            opt.step()
            opt.zero_grad()

MNISTLogistic.fit = fit 

In [48]:
model = MNISTLogistic()

model.fit(train_dataloader)

print(loss_func(model(xb), yb))

tensor(0.3974, grad_fn=<NllLossBackward0>)


### Swapping in another model

In [49]:
from text_recognizer.models.mlp import MLP 

MLP.fit = fit

In [50]:
MLP.forward??

In [51]:
MLP.__init__??

In [52]:
MLP?

In [53]:
digits_to_9 = list(range(10))
data_config = {"input_dims": (784,), "mapping": {digit: str(digit) for digit in digits_to_9}}
data_config

{'input_dims': (784,),
 'mapping': {0: '0',
  1: '1',
  2: '2',
  3: '3',
  4: '4',
  5: '5',
  6: '6',
  7: '7',
  8: '8',
  9: '9'}}

In [54]:
model = MLP(data_config)
model

MLP(
  (fc1): Linear(in_features=784, out_features=1024, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=1024, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)

In [55]:
model.fc1.weight

Parameter containing:
tensor([[ 0.0114, -0.0035,  0.0036,  ..., -0.0055, -0.0151,  0.0256],
        [-0.0276,  0.0085,  0.0221,  ...,  0.0091,  0.0206, -0.0302],
        [ 0.0059, -0.0135,  0.0255,  ...,  0.0336, -0.0219,  0.0240],
        ...,
        [ 0.0311, -0.0077, -0.0144,  ..., -0.0324,  0.0316, -0.0280],
        [-0.0300,  0.0034, -0.0024,  ...,  0.0304,  0.0138, -0.0331],
        [-0.0273, -0.0180,  0.0299,  ...,  0.0354,  0.0094,  0.0227]],
       requires_grad=True)

In [56]:
%%time

print("before training: ", loss_func(model(xb), yb))

train_ds = BaseDataset(x_train, y_train)
train_dataloader = DataLoader(train_ds, batch_size=bs)
fit(model, train_dataloader)

print("after training: ", loss_func(model(xb), yb))

before training:  tensor(2.2985, grad_fn=<NllLossBackward0>)
after training:  tensor(0.1131, grad_fn=<NllLossBackward0>)
CPU times: user 20.1 s, sys: 244 ms, total: 20.3 s
Wall time: 21.7 s


## Extra goodies: data organization, validation, and acceleration


### Making the GPU go brrrr

In [57]:
if torch.cuda.is_available():
    !nvidia-smi
else:
    print(":(")

Fri Aug 26 00:52:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [58]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model.to(device)

loss_func(model(xb.to(device)), yb.to(device))

device

device(type='cuda')

In [59]:
def push_to_device(tensor):
    return tensor.to(device)

train_ds = BaseDataset(x_train, y_train, transform=push_to_device, target_transform=push_to_device)

train_dataloader = DataLoader(train_ds, batch_size=bs)

In [60]:
for xb, yb in train_dataloader:
    print(xb.device, yb.device)
    break;

cuda:0 cuda:0


In [61]:
%%time

model = MLP(data_config)
model.to(device)

model.fit(train_dataloader)

print(loss_func(model(push_to_device(xb)), push_to_device(yb)))

tensor(0.1047, device='cuda:0', grad_fn=<NllLossBackward0>)
CPU times: user 6.55 s, sys: 112 ms, total: 6.66 s
Wall time: 6.76 s


## Adding validation data and organizing data code with `DataModule`

In [81]:
class MNISTDataModule:
    url = "https://github.com/pytorch/tutorials/raw/master/_static/"
    filename = "mnist.pkl.gz"

    def __init__(self, dir, bs=32):
        self.dir = dir
        self.bs = bs
        self.path = self.dir / self.filename

    def prepare_data(self):
        if not (self.path).exists():
            content = requests.get(self.url + self.filename).content
            self.path.open("wb").write(content)

    def setup(self):
        with gzip.open(self.path, "rb") as f:
            ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")

        x_train, y_train, x_valid, y_valid = map(
            torch.Tensor, (x_train, y_train, x_valid, y_valid)
        )
        # print(y_train[0], y_train[0].type)
        # print(y_valid[0], y_valid[0].type)

        self.train_ds = BaseDataset(x_train, y_train, transform=push_to_device, target_transform=push_to_device)
        self.valid_ds = BaseDataset(x_valid, y_valid, transform=push_to_device, target_transform=push_to_device)

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_ds, batch_size=self.bs, shuffle=True)

    def valid_dataloader(self):
        return torch.utils.data.DataLoader(self.valid_ds, batch_size=self.bs, shuffle=True)

In [99]:
def fit(self: nn.Module, datamodule):
    datamodule.prepare_data()
    datamodule.setup()

    val_dataloader = datamodule.valid_dataloader()

    self.eval()
    with torch.no_grad():
        valid_loss = sum(loss_func(self(xb), yb.type(torch.LongTensor).to(device)) for xb, yb in val_dataloader)

    print("before start of training: ", valid_loss / len(val_dataloader))

    opt = configure_optimizer(self)
    train_dataloader = datamodule.train_dataloader()

    for epoch in range(epochs):
        self.train()

        for xb, yb in train_dataloader:
            pred = self(xb)
            loss = loss_func(pred, yb.type(torch.LongTensor).to(device))

            loss.backward()
            opt.step()
            opt.zero_grad()

        self.eval()
        with torch.no_grad():
            valid_loss = sum(loss_func(self(xb), yb.type(torch.LongTensor).to(device)) for xb, yb in val_dataloader)
        print(epoch, valid_loss / len(val_dataloader))

MNISTLogistic.fit = fit
MLP.fit = fit

In [100]:
model = MLP(data_config)
model.to(device)

datamodule = MNISTDataModule(dir=path, bs=32)

model.fit(datamodule=datamodule)

before start of training:  tensor(2.3064, device='cuda:0')
0 tensor(0.1637, device='cuda:0')
1 tensor(0.1130, device='cuda:0')


### try out different hyperparameters for the `MLP` and for training

In [101]:
MLP.__init__??

In [108]:
%%time

from argparse import Namespace 

args = Namespace(epochs=2, bs=32) 

epochs = 8
bs = 128 

def configure_optimizer(model: nn.Module) -> optim.Optimizer:
    return optim.Adam(model.parameters(), lr=3e-4)

model = MLP(data_config, args=args)
model.to(device)

datamodule = MNISTDataModule(dir=path, bs=bs)

model.fit(datamodule=datamodule)

before start of training:  tensor(2.3041, device='cuda:0')
0 tensor(0.2283, device='cuda:0')
1 tensor(0.1610, device='cuda:0')
2 tensor(0.1271, device='cuda:0')
3 tensor(0.1105, device='cuda:0')
4 tensor(0.0971, device='cuda:0')
5 tensor(0.0872, device='cuda:0')
6 tensor(0.0801, device='cuda:0')
7 tensor(0.0790, device='cuda:0')
CPU times: user 23.8 s, sys: 255 ms, total: 24 s
Wall time: 24 s


In [109]:
val_dataloader = datamodule.valid_dataloader()
valid_acc = sum(accuracy(model(xb), yb) for xb, yb in val_dataloader) / len(val_dataloader)
valid_acc

tensor(0.9770, device='cuda:0')